In [13]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [ ]:
# below is the esp8266 version
#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# You need the esp8266 code built with mqtt_as libraries
# See https://github.com/goatchurchprime/jupyter_micropython_developer_notebooks/blob/master/mqtt_async_projects/esp8266_commissioning_mqtt_as.ipynb

In [3]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 52 lines (1860 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [14]:
%sendtofile config.txt

wifiname        DoESLiverpool
wifipassword    decafbad00
mqttbroker      test.mosquitto.org
pinled          2
boardname       miniwheels1
mAen            13
mAfore          0
mAback          16
mBen            15
mBfore          14
mBback          12

Sent 12 lines (303 bytes) to config.txt.


In [17]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [18]:
%sendtofile main.py

import os, time, sys, machine, itertools, time
from machine import Pin, PWM
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from mqtt_as import MQTTClient

from stdmqttas import pinled, flashpinled, shortmac
#flashpinled(5, 300, 300)
import network
network.WLAN().active(0)  # disable the connection at startup


from machine import Pin, PWM
mApwm = PWM(Pin(int(fconfig["mAen"])))
mAfore = Pin(int(fconfig["mAfore"]), Pin.OUT)
mAback = Pin(int(fconfig["mAback"]), Pin.OUT)
mApwm.freq(10)
mApwm.duty(0)

mBpwm = PWM(Pin(int(fconfig["mBen"])))
mBfore = Pin(int(fconfig["mBfore"]), Pin.OUT)
mBback = Pin(int(fconfig["mBback"]), Pin.OUT)
mBpwm.freq(10)
mBpwm.duty(0)

timeZeroBy = 1
async def zeroingtask():
    global timeZeroBy
    while True:
        if timeZeroBy != 0:
            if time.ticks_ms() > timeZeroBy:
                timeZeroBy = 0
                mAfore.value(0)
                mAback.value(0)
                mBfore.value(0)
                mBback.value(0)
                pinled.value(1)
        await asyncio.sleep_ms(50)


def updatewheels(msg):
    global timeZeroBy
    try:
        s = msg.split()
        timeZeroBy = time.ticks_ms() + int(s[6])
        mApwm.duty(int(s[0]))
        mAfore.value(int(s[1]))
        mAback.value(int(s[2]))
        mBpwm.duty(int(s[3]))
        mBfore.value(int(s[4]))
        mBback.value(int(s[5]))
        pinled.value(0)
    except Exception as e:
        print("Bad wheels", msg, e)


topicstem = fconfig["boardname"].encode()
topicboardcmd = topicstem + b"/cmd"
topicboardwheels = topicstem + b"/wheels"
topicstatus = topicstem+b"/status"

print("topicstem", topicstem)
import network
network.WLAN().active(0)  # disable the connection at startup

for l in ["500 1 0 500 1 0 1", "500 0 1 500 0 1 1", 
          "500 1 0 500 0 1 1", "500 0 1 500 1 0 1", "500 0 0 500 0 0 1"]:
    updatewheels(l)
    time.sleep_ms(500)

def callbackcmd(topic, msg, retained):
    print(topic, msg)
    if topic == topicboardcmd:
        aloop.create_task(callbackcmdtask(client, topicreply, msg))
    else:
        updatewheels(msg)

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    await client.publish(topicstatus, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    
    
async def onconnecttask(client):
    print("subscribing")
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topicstatus, ipnumber, retain=True)
    print("subscribing to", topicboardcmd, topicboardwheels)
    await client.subscribe(topicboardcmd, 1)
    await client.subscribe(topicboardwheels, 1)
    

config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topicstatus, "offline", True)
client = MQTTClient(config)
client.DEBUG = True
  
aloop = asyncio.get_event_loop()
aloop.create_task(zeroingtask())
aloop.create_task(mqttconnecttask(client))
aloop.run_forever()
    

Sent 104 lines (2976 bytes) to main.py.


In [ ]:
# leftover attempts at light controls

from machine import Pin, I2C
i2c = I2C(scl=Pin(15), sda=Pin(4), freq=450000)
print(list(map(hex, i2c.scan())))
# TCS3472 
i2c.writeto_mem(0x29, 0x80, b'\x03')
print(hex(i2c.readfrom_mem(0x29, 0x92, 1)[0]))
x = i2c.readfrom_mem(0x29, 0x8F, 1)  # gain of 1x
print(x)
x = i2c.readfrom_mem(0x29, 0x81, 1)
print(x)
print(hex(i2c.readfrom_mem(0x29, 0x92, 1)[0]))
i2c.writeto_mem(0x29, 0x8F, b'\x00')  # gain of 1x
i2c.writeto_mem(0x29, 0x81, b'\x00')  # integration time 700ms

i2c.writeto_mem(0x29, 0x8F, b'\x01')  # gain of 2x
i2c.writeto_mem(0x29, 0x81, b'\xC0')  # integration time 154ms

i2c.writeto_mem(0x29, 0x8F, b'\x02')  # gain of 4x
i2c.writeto_mem(0x29, 0x81, b'\xEB')  # integration time 50ms

import struct, time
t0 = time.ticks_ms()
for i in range(20):
    print(time.ticks_ms() - t0, struct.unpack("<HHHH", i2c.readfrom_mem(0x29, 0x94, 8)))
    time.sleep(0.01)